# Test of Neural Networks on the Elzermann readout data

In [1]:
import os, pickle

import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt


import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

# Colors using scheme
c = ["#" + i for i  in "264653-2a9d8f-e9c46a-f4a261-e76f51".split("-")]
from sklearn.metrics import roc_curve, roc_auc_score


def fft_abs_phase(input):
    transformed = np.fft.rfft(input)
    output      = np.concatenate([np.abs(transformed), np.cos(np.angle(transformed)), np.sin(np.angle(transformed))],
                                  axis = 1)
    return output
X_train, X_test, y_train, y_test = pd.read_pickle("/mnt/c/Users/johan/Desktop/CMT_project/data/train_test_data.dat")

2022-02-19 13:39:58.671941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 13:39:58.672099: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Imports

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.metrics import AUC, BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from keras_lr_finder import LRFinder

import tensorflow.keras.backend as K


from sklearn.preprocessing import RobustScaler

### Common training parameters

In [16]:
epochs     = 50
batch_size = 256
timesteps  = 220
val_split  = 0.10
patience   = 10

scaler              = RobustScaler()
X_train_scaled      = scaler.fit_transform(X_train)
X_test_scaled       = scaler.transform(X_test)

## Feed-Forward

In [4]:
# Parameters
name           = "feed_forward_network"
dropout        = 0.25
hidden_states  = 64 
learning_rate  = 5e-3
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x     = Dense(hidden_states, activation = "relu")(input)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)

model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


2022-02-19 13:40:05.863908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-19 13:40:05.863961: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-19 13:40:05.863980: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-B87RRKQ): /proc/driver/nvidia/version does not exist
2022-02-19 13:40:05.864220: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 220)]             0         
                                                                 
 dense (Dense)               (None, 64)                14144     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0     

## 1D-conv:


In [5]:
from tensorflow.keras.layers import Conv1D, Flatten, Normalization
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.preprocessing import QuantileTransformer

lr_schedule_func = lambda epoch, lr: lr if epoch < 5 else 0.90 * lr

# qnt_scaler = QuantileTransformer()
# qnt_scaler.fit(X_train)

# norm_layer = Normalization(mean = tf.convert_to_tensor(scaler.center_, dtype = tf.float32), variance = tf.convert_to_tensor(scaler.scale_ ** 2, dtype = tf.float32))
norm_layer = Normalization()
norm_layer.adapt(X_train)

In [58]:
# Parameters
name           = "1D_conv"
dropout        = 0.5
filter_sizes   = (32,  16, 16, 8)
kernal_sizes   = (101, 51, 25, 10)
hidden_states  = 32
learning_rate  = 5e-4
test_lr        = False
test_interval  = (1e-9, 10)
fit_model      = True
test_model     = True
save_preds     = True
save_model     = True


input = Input(shape = (timesteps))

x = norm_layer(input)

x = tf.expand_dims(x, -1)

for fs, ks in zip(filter_sizes, kernal_sizes):
    x     = Conv1D(fs, kernel_size = ks, activation = "relu")(x)

x     = Flatten()(x)

# x     = tf.squeeze(x, -1)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = [LearningRateScheduler(lr_schedule_func, verbose = 0),
                     EarlyStopping("val_loss", patience = patience, restore_best_weights = True)]
        )

if test_model:
    model.evaluate(X_test, y_test)

    test_preds = model(X_test)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)

if save_model:
    model.save(f"../models/{name}")

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 220)]             0         
                                                                 
 normalization_16 (Normaliza  (None, 220)              441       
 tion)                                                           
                                                                 
 tf.expand_dims_27 (TFOpLamb  (None, 220, 1)           0         
 da)                                                             
                                                                 
 conv1d_102 (Conv1D)         (None, 120, 32)           3264      
                                                                 
 conv1d_103 (Conv1D)         (None, 70, 16)            26128     
                                                                 
 conv1d_104 (Conv1D)         (None, 46, 16)            641

In [ ]:
model.save(f"../models/{name}")

## Attention

In [6]:
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization

In [17]:
def transformer_block(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = Dropout(dropout)(x)
    x = Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    return x + res


In [18]:
# Parameters
name           = "Attention"
dropout        = 0.25
# filter_sizes   = (32,  16, 16, 8)
# kernal_sizes   = (101, 51, 25, 10)
attn_heads     = 8
key_dim        = 8


hidden_states  = 32
learning_rate  = 1e-2
test_lr        = False
test_interval  = (1e-9, 10)
fit_model      = True
test_model     = True
save_preds     = False
save_model     = False


input = Input(shape = (timesteps))
input = norm_layer(input)
input = tf.expand_dims(input, -1)
x     = input

for _ in range(3):
    x = transformer_block(x, head_size = key_dim, num_heads = attn_heads, ff_dim = hidden_states, dropout = dropout)

x     = Flatten()(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = [LearningRateScheduler(lr_schedule_func, verbose = 0),
                     EarlyStopping("val_loss", patience = patience, restore_best_weights = True)]
        )

if test_model:
    model.evaluate(X_test, y_test)

    test_preds = model(X_test)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)

if save_model:
    model.save(f"../models/{name}")

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 220, 1)]     0           []                               
                                                                                                  
 multi_head_attention_18 (Multi  (None, 220, 1)      449         ['input_18[0][0]',               
 HeadAttention)                                                   'input_18[0][0]']               
                                                                                                  
 dropout_48 (Dropout)           (None, 220, 1)       0           ['multi_head_attention_18[1][0]']
                                                                                                  
 layer_normalization_33 (LayerN  (None, 220, 1)      2           ['dropout_48[1][0]']       

KeyboardInterrupt: 

In [ ]:
model.save(f"../models/{name}")

## LSTM: 


In [71]:
from tensorflow.keras.layers import LSTM

In [75]:
# Parameters
name           = "LSTM"
dropout        = 0.15
hidden_states  = 32 
units          = (4, 8, 4)
learning_rate  = 1e-2
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x = input
x = tf.expand_dims(input, -1)

for u in units:
    x     = LSTM(u, dropout = dropout, return_sequences = True)(x)

x     = Flatten()(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


Model: "model_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_55 (InputLayer)       [(None, 220)]             0         
                                                                 
 tf.expand_dims_13 (TFOpLamb  (None, 220, 1)           0         
 da)                                                             
                                                                 
 lstm_9 (LSTM)               (None, 220, 4)            96        
                                                                 
 lstm_10 (LSTM)              (None, 220, 8)            416       
                                                                 
 lstm_11 (LSTM)              (None, 220, 4)            208       
                                                                 
 flatten_6 (Flatten)         (None, 880)               0         
                                                          

## TCN

In [77]:
from tcn import TCN, tcn_full_summary

In [92]:
# Parameters
name           = "TCN"
dropout        = 0.25
hidden_states  = 32
nb_filters     = 8
kernel_size    = 3
dilations      = (1, 2, 4, 8, 16, 32) 
learning_rate  = 1e-3
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x     = input
x     = tf.expand_dims(input, -1)

x     = TCN(
    nb_filters  = nb_filters,
    kernel_size = kernel_size,
    dilations   = dilations,
    dropout_rate = dropout,
    return_sequences = True)(x)

x     = Flatten()(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


Model: "model_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_68 (InputLayer)       [(None, 220)]             0         
                                                                 
 tf.expand_dims_26 (TFOpLamb  (None, 220, 1)           0         
 da)                                                             
                                                                 
 tcn_16 (TCN)                (None, 220, 8)            2248      
                                                                 
 flatten_18 (Flatten)        (None, 1760)              0         
                                                                 
 dense_193 (Dense)           (None, 32)                56352     
                                                                 
 dropout_151 (Dropout)       (None, 32)                0         
                                                          